xml version: 0ecdc07

In [5]:
xmlFile = 'MTMount_Telemetry.xml'
partialxml = 'Camera_Cable_Wrap'
partialxml = partialxml.lower()

In [6]:
import numpy as np
from astropy.time import Time
from datetime import timedelta, datetime
from lsst_efd_client import EfdClient

import pandas as pd

pd.set_option('display.max_rows', None)

In [7]:
client = EfdClient('summit_efd')
csc_index = 1

#start = Time('2020-11-03T11:41:30') #this is UTC
#end = start+timedelta(days=1)
#end = Time(datetime.now())
end = Time('2020-11-11T14:40:30') #this is UTC
start = end - timedelta(seconds=5)
#start = end - timedelta(days=5)

#import astropy
#astropy.utils.data.clear_download_cache()

In [8]:
fid = open(xmlFile)

topics = dict()
params = dict()
counts = dict()
lastT = dict()
values= dict()
nline = 0
for line in fid:
    if '<EFDB_Topic>' in line:
        validTopic = False
        i = line.find('<EFDB')
        topic = line[i+12:-14]
        t1, t2 = topic.split('_',1)
        if 'partialxml' in globals() and t2.lower() in partialxml:
            validTopic = True
            dfa = await client.select_time_series('lsst.sal.%s.%s'%(t1,t2), '*', 
                                     (start-timedelta(seconds=37)).tai, (end-timedelta(seconds=37)).tai, csc_index)
    elif '<EFDB_Name>' in line:
        i = line.find('<EFDB')
        param = line[i+11:-13]
        if validTopic:
            nline+=1
            idx = '%d'%nline
    elif '<Count>' in line:
        i = line.find('<C')
        count = line[i+7:-9]
        
        if validTopic:
            print(topic, param,count)
            topics[idx] = topic
            params[idx] = param
            counts[idx] = count
            c = int(count)
            try:
                lastT[idx] = pd.to_datetime(dfa.private_sndStamp[-1], unit='s')
                if c == 1:
                    values[idx] = dfa[param][-1]
                else:
                    aa = [dfa['%s%d'%(param,i)][-1] for i in range(c)]
                    values[idx] = np.mean(aa)
                    if np.isnan(values[idx]):
                        print(aa)
            except:
                values[idx] = np.nan
    #if nline>7:
    #    break
print(nline)
fid.close()

MTMount_Camera_Cable_Wrap CCW_Status 1
MTMount_Camera_Cable_Wrap CCW_Status_Drive_1 1
MTMount_Camera_Cable_Wrap CCW_Status_Drive_2 1
MTMount_Camera_Cable_Wrap AXES_PXI_Version 1
MTMount_Camera_Cable_Wrap TMA_PXI_Version 1
MTMount_Camera_Cable_Wrap CCW_Angle_1 1
MTMount_Camera_Cable_Wrap CCW_Angle_2 1
MTMount_Camera_Cable_Wrap CCW_Rotator_Position 1
MTMount_Camera_Cable_Wrap CCW_Speed_1 1
MTMount_Camera_Cable_Wrap CCW_Speed_2 1
MTMount_Camera_Cable_Wrap CCW_Current_1 1
MTMount_Camera_Cable_Wrap CCW_Current_2 1
MTMount_Camera_Cable_Wrap sub_Rotator_Position 1
MTMount_Camera_Cable_Wrap CCW_Positive_Directional_limit_switch 1
MTMount_Camera_Cable_Wrap CCW_Negative_Directional_limit_switch 1
MTMount_Camera_Cable_Wrap CCW_Interlocks 10
MTMount_Camera_Cable_Wrap timestamp 1
17


In [9]:
frame = {'Topic': topics, 'Name': params,
         'Count': counts, 'Last Timestamp (UTC)': lastT,
         'value (average)': values}
d = pd.DataFrame(frame)

In [10]:
d

,Topic,Name,Count,Last Timestamp (UTC),value (average)
1,MTMount_Camera_Cable_Wrap,CCW_Status,1,2020-11-11 14:40:29.914482594,On\Enable
2,MTMount_Camera_Cable_Wrap,CCW_Status_Drive_1,1,2020-11-11 14:40:29.914482594,Off
3,MTMount_Camera_Cable_Wrap,CCW_Status_Drive_2,1,2020-11-11 14:40:29.914482594,Standstill
4,MTMount_Camera_Cable_Wrap,AXES_PXI_Version,1,2020-11-11 14:40:29.914482594,None
5,MTMount_Camera_Cable_Wrap,TMA_PXI_Version,1,2020-11-11 14:40:29.914482594,1.0 CCWAux
6,MTMount_Camera_Cable_Wrap,CCW_Angle_1,1,2020-11-11 14:40:29.914482594,-49.9629
7,MTMount_Camera_Cable_Wrap,CCW_Angle_2,1,2020-11-11 14:40:29.914482594,-50
8,MTMount_Camera_Cable_Wrap,CCW_Rotator_Position,1,2020-11-11 14:40:29.914482594,-0.0005779
9,MTMount_Camera_Cable_Wrap,CCW_Speed_1,1,2020-11-11 14:40:29.914482594,0
10,MTMount_Camera_Cable_Wrap,CCW_Speed_2,1,2020-11-11 14:40:29.914482594,0
